# Part 1: File Preparation

In [1]:
import pandas as pd

## read sample file in in chunks, and print the column names

In [25]:
reader = pd.read_csv("sample_data.csv", sep=';', chunksize=10000)

chunk = next(reader)
for c in chunk.columns:
    print(c)

user.Id
user.MobileNumber
user.FaxNumber
user.Identifier
sentToMTA.Timestamp
sentToMTA.ExternalTransactionId
sentToMTA.SendoutChannel
skip.Timestamp
skip.Category
skip.ExternalTransactionId
bounce.Timestamp
bounce.Category
feedback.Timestamp
feedback.Category
render.Timestamp
render.GeoLocation.ISOCountryCode
render.GeoLocation.RegionCode
render.GeoLocation.City
render.GeoLocation.Latitude
render.GeoLocation.Longitude
render.UserAgent.Platform
render.UserAgent.OperatingSystemName
render.UserAgent.OperatingSystemVersion
render.UserAgent.ClientName
render.UserAgent.ClientVersion
render.UserAgent.RawString
click.Timestamp
click.Link.Id
click.Link.Category.Id
click.Link.Category.Name
click.Link.Url
click.Link.SystemLinkType
click.Link.Type
click.GeoLocation.ISOCountryCode
click.GeoLocation.RegionCode
click.GeoLocation.City
click.GeoLocation.Latitude
click.GeoLocation.Longitude
click.UserAgent.Platform
click.UserAgent.OperatingSystem
click.UserAgent.OperatingSystemVersion
click.UserAgent.Cl

## Small Dataset

- drop all rows where column `click.Timestamp` is N/A
- concatenate the remaining rows to a single dataframe

In [47]:
reader = pd.read_csv("sample_data.csv", sep=';', chunksize=10000)

df_clicked = pd.DataFrame()
for chunk in reader:
    df = chunk.dropna(subset=['click.Timestamp'])

    df_clicked = pd.concat([df_clicked, df])

df_clicked.shape

(346, 117)

## Large Dataset

do the same for a large dataset

In [55]:
reader = pd.read_csv("sample_data_large.csv", sep=';', chunksize=10000)

df_clicked = pd.DataFrame()
for chunk in reader:
    df = chunk.dropna(subset=['click.Timestamp'])
    df_clicked = pd.concat([df_clicked, df])

df_clicked.shape

(47301, 118)

write the result to a file

In [56]:
df_clicked = df_clicked.dropna(axis=1, how='all')
df_clicked.to_csv('df_clicked.csv',sep=';')

In [57]:
df_clicked.shape

(47301, 75)

In [38]:
df2 = pd.concat([chunk, df])
df2

,user.Id,user.MobileNumber,user.FaxNumber,user.Identifier,sentToMTA.Timestamp,sentToMTA.ExternalTransactionId,sentToMTA.SendoutChannel,skip.Timestamp,skip.Category,skip.ExternalTransactionId,...,user.CustomAttribute['SCS_Due_Date'],user.CustomAttribute['SCS_Entry_Date'],user.CustomAttribute['SCS_Expiry_Dat'],user.CustomAttribute['SCS_Magazine_Language_ISO'],user.CustomAttribute['SSFCardNumber'],user.CustomAttribute['Street'],user.CustomAttribute['Street2'],user.CustomAttribute['Street3'],user.CustomAttribute['Telephone'],user.CustomAttribute['TitleKey']
0,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-09T08:30:26Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-06T11:48:44Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-05T08:30:18Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-03T08:30:21Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-26T11:30:21Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-23T11:30:20Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-19T08:45:12Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-18T11:30:16Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-11T11:30:20Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-10T16:00:30Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df = next(reader)
df

C:\Users\BREIBI\AppData\Local\Programs\Python\Python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (38,39,40,93,103,104,106,107,108,109,110,112) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user.Id,user.MobileNumber,user.FaxNumber,user.Identifier,sentToMTA.Timestamp,sentToMTA.ExternalTransactionId,sentToMTA.SendoutChannel,skip.Timestamp,skip.Category,skip.ExternalTransactionId,...,user.CustomAttribute['SCS_Due_Date'],user.CustomAttribute['SCS_Entry_Date'],user.CustomAttribute['SCS_Expiry_Dat'],user.CustomAttribute['SCS_Magazine_Language_ISO'],user.CustomAttribute['SSFCardNumber'],user.CustomAttribute['Street'],user.CustomAttribute['Street2'],user.CustomAttribute['Street3'],user.CustomAttribute['Telephone'],user.CustomAttribute['TitleKey']
20000,950244360,NaN,NaN,NaN,2015-04-18T13:01:36Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20001,950244360,NaN,NaN,NaN,2015-04-16T13:02:28Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20002,950244360,NaN,NaN,NaN,2015-04-13T13:00:33Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20003,950244360,NaN,NaN,NaN,2015-04-09T13:01:44Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20004,950244360,NaN,NaN,NaN,2015-04-07T13:01:41Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20005,950244360,NaN,NaN,NaN,2015-04-02T13:01:39Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20006,950244360,NaN,NaN,NaN,2015-03-31T13:02:55Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20007,950244360,NaN,NaN,NaN,2015-03-29T13:01:40Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20008,950244360,NaN,NaN,NaN,2015-03-26T14:02:30Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20009,950244360,NaN,NaN,NaN,2015-03-25T14:01:35Z,NaN,EMAIL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df2.shape

(10000, 117)

In [42]:
df2 = pd.concat([chunk, df])
df2

,user.Id,user.MobileNumber,user.FaxNumber,user.Identifier,sentToMTA.Timestamp,sentToMTA.ExternalTransactionId,sentToMTA.SendoutChannel,skip.Timestamp,skip.Category,skip.ExternalTransactionId,...,user.CustomAttribute['SCS_Due_Date'],user.CustomAttribute['SCS_Entry_Date'],user.CustomAttribute['SCS_Expiry_Dat'],user.CustomAttribute['SCS_Magazine_Language_ISO'],user.CustomAttribute['SSFCardNumber'],user.CustomAttribute['Street'],user.CustomAttribute['Street2'],user.CustomAttribute['Street3'],user.CustomAttribute['Telephone'],user.CustomAttribute['TitleKey']
0,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-09T08:30:26Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-06T11:48:44Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-05T08:30:18Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-03T08:30:21Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-26T11:30:21Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-23T11:30:20Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-19T08:45:12Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-18T11:30:16Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-11T11:30:20Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,950228553,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-10T16:00:30Z,SYS_BOUNCED,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df2.shape

(20000, 117)

In [ ]:
#chunk1 = chunk[chunk['']]
#for i, chunk in enumerate(reader):
#    print(chunk)

In [27]:
chunk.shape

(10000, 117)

In [30]:
chunk2 = chunk.dropna(subset=['click.Timestamp'])
chunk2.shape

(16, 117)

In [34]:
chunk3 = chunk2.dropna(axis=1, how='all')
chunk3.shape

(16, 67)

In [35]:
chunk3

,user.Id,user.Identifier,sentToMTA.SendoutChannel,click.Timestamp,click.Link.Id,click.Link.Category.Id,click.Link.Category.Name,click.Link.Url,click.Link.Type,click.GeoLocation.ISOCountryCode,...,user.CustomAttribute['SCS_Boutique_City'],user.CustomAttribute['SCS_Boutique_Name1'],user.CustomAttribute['SCS_Boutique_Street'],user.CustomAttribute['SCS_Due_Date'],user.CustomAttribute['SCS_Entry_Date'],user.CustomAttribute['SCS_Expiry_Dat'],user.CustomAttribute['SCS_Magazine_Language_ISO'],user.CustomAttribute['SSFCardNumber'],user.CustomAttribute['Street'],user.CustomAttribute['TitleKey']
374,950238665,NaN,EMAIL,2015-01-12T12:35:55Z,43.0,0.0,0.0,https://www.swarovski.com/newsletter?subscribe...,NORMAL,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1677,950231552,NaN,EMAIL,2015-01-29T05:46:19Z,22.0,0.0,0.0,http://www.swarovski.com/91109/promotion?Banne...,NORMAL,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4149,950232143,NaN,EMAIL,2015-12-28T17:37:24Z,24.0,0.0,0.0,http://www.swarovski.com/20/category?BannerID=...,NORMAL,LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4175,950232143,NaN,EMAIL,2015-11-12T14:10:11Z,25.0,0.0,0.0,http://www.swarovski.com/1708/category?BannerI...,NORMAL,LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4176,950232143,NaN,EMAIL,2015-11-12T14:09:55Z,25.0,0.0,0.0,http://www.swarovski.com/1708/category?BannerI...,NORMAL,LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6206,950228712,NaN,EMAIL,2015-12-29T14:09:39Z,23.0,0.0,0.0,http://www.swarovski.com/sale?BannerID=0800403...,NORMAL,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6284,950237474,C014994206,EMAIL,2015-11-11T19:34:16Z,41.0,0.0,0.0,http://www.swarovski.com?utm_campaign=151110_e...,NORMAL,A2,...,WHITE PLAINS,SWAROVSKI RETAIL VENTURES #208,125 WESTCHESTER AVE SPC 3780,31.12.2020,05.12.1987,Dez.20,EN,1.010922e+10,28 Patton Dr,1.0
6582,950238895,C015007242,EMAIL,2015-12-10T11:22:09Z,25.0,0.0,0.0,http://www.swarovski.com/1703/category?BannerI...,NORMAL,BE,...,WAREGEM,Berenice,Stationsstraat 58,30.06.2020,01.07.1991,Jun.20,NL,NaN,Baanstknokstraat 60,1.0
6595,950238895,C015007242,EMAIL,2015-11-25T11:36:49Z,23.0,0.0,0.0,http://www.swarovski.com/91281/promotion?Banne...,NORMAL,BE,...,WAREGEM,Berenice,Stationsstraat 58,30.06.2020,01.07.1991,Jun.20,NL,NaN,Baanstknokstraat 60,1.0
6679,950232143,NaN,EMAIL,2015-10-06T19:56:09Z,52.0,0.0,0.0,http://www.swarovski.com/01/category?BannerID=...,NORMAL,LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
